# Generación de Data Única referente a Duraciones

Transformación de algunos datos para normalizar nombres en columnas

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from unicodedata import normalize

from pjud.data import cleandata

In [2]:
tqdm.pandas()

path_interim = "../data/interim/pjud"
path_processed = "../data/processed/pjud"

/Users/jalvaradoruiz/opt/anaconda3/envs/py39/lib/python3.9/site-packages/tqdm/std.py:699: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


# DATA de TRIBUNALES OBTENIDAS DEL COT

In [4]:
# Analizo contra los datos extraidos en COT
df_tribunales = pd.read_feather(f"{path_processed}/generates_ListadoTribunales.feather")
tribunales = df_tribunales['TRIBUNAL'].unique()

# DATA DURACION


In [5]:
df_duracion = pd.read_feather(f"{path_interim}/clean_Duraciones.feather")
df_duracion.sort_values('FECHA INGRESO')

,index,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,FECHA INGRESO,FECHA TERMINO,DURACIÓN CAUSA,MOTIVO TERMINO,MES TERMINO,AÑO TERMINO,TOTAL TERMINOS
974678,299951,46,C.A. DE CONCEPCION,1080,JUZGADO DE GARANTIA DE TALCAHUANO,2398-2017,Ordinaria,1900-01-01,2018-05-31,43249,DECLARA SOBRESEIMIENTO DEFINITIVO.,may-18,2018,1
1075778,401124,56,C.A. DE PUERTO MONTT,662,JUZGADO DE LETRAS Y GARANTIA DE QUELLON,684-2018,Ordinaria,1900-01-01,2018-08-21,43331,SENTENCIA,ago-18,2018,1
1012613,337911,50,C.A. DE TEMUCO,214,JUZGADO DE LETRAS Y GARANTIA DE PUCON,1059-2017,Ordinaria,1900-01-01,2018-10-01,43372,DECLARA SOBRESEIMIENTO DEFINITIVO.,oct-18,2018,1
2766444,91242,25,C.A. DE LA SERENA,932,JUZGADO DE GARANTIA DE OVALLE,1-2000,Ordinaria,2000-12-20,2015-06-02,5277,CERTIFICA CUMPLIMIENTO ART. 468.,jun-15,2015,1
2768930,93728,25,C.A. DE LA SERENA,932,JUZGADO DE GARANTIA DE OVALLE,3-2000,Ordinaria,2000-12-21,2015-06-02,5276,NO PERSEVERAR EN EL PROCEDIMIENTO.,jun-15,2015,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399265,399530,55,C.A. DE VALDIVIA,1086,JUZGADO DE GARANTIA DE LOS LAGOS,1756-2019,Ordinaria,2019-12-31,2019-12-31,0,ACOGE REQUERIMIENTO (MONITORIO).,dic-19,2019,1
399264,399529,55,C.A. DE VALDIVIA,1086,JUZGADO DE GARANTIA DE LOS LAGOS,1753-2019,Ordinaria,2019-12-31,2019-12-31,0,APROBACION NO INICIO INVESTIGACION.,dic-19,2019,1
600818,601268,91,C.A. DE SAN MIGUEL,1230,11º JUZGADO DE GARANTIA DE SANTIAGO,10306-2019,Ordinaria,2019-12-31,2019-12-31,0,COMUNICA Y/O APLICA DECISION PPIO. DE OPORTUNI...,dic-19,2019,1
399270,399535,55,C.A. DE VALDIVIA,1086,JUZGADO DE GARANTIA DE LOS LAGOS,1761-2019,Ordinaria,2019-12-31,2019-12-31,0,APROBACION NO INICIO INVESTIGACION.,dic-19,2019,1


In [6]:
filtro_null = df_duracion[df_duracion['FECHA INGRESO'].isnull()]
print(f"Existen {len(filtro_null)} causas sin fecha de ingresos")

Existen 0 causas sin fecha de ingresos


In [7]:
df_duracion.drop(filtro_null.index, axis=0, inplace=True)

In [8]:
# Normalizo nombres

df_duracion['TRIBUNAL'] = df_duracion['TRIBUNAL'].progress_apply(cleandata.cambio_nombre_juzgados)

100%|██████████| 3354732/3354732 [00:14<00:00, 224010.74it/s]


In [9]:
# Verifico si coinciden los nombres e imprimo los que estan en conflicto

for tribunal in df_duracion['TRIBUNAL'].unique():
    if not tribunal in tribunales:
        print(tribunal)

## CARGA DATA A ARCHIVOS FUTHER

In [10]:
# Directorio donde se guardaran archivos feather
df_duracion.reset_index(inplace=True, drop=True)
os.makedirs(path_processed, exist_ok = True)  
# Guardamos dataset como archivo feather
df_duracion.to_feather(f'{path_processed}/processes_Duraciones.feather')